O que é o DES?
Contextualizar
Comparar com a cifra de Feistel
Falar sobre confusão e difusão
Explicar como ele funciona (black box)

In [ ]:
from typing import List


def feistel_encrypt(plaintext: int, key: int) -> int:
    pass


def permutate_bits(bits: int) -> int:
    pass


def generate_subkeys(key: int) -> List[int]:
    pass


def left_circular_shift(bits: int) -> int:
    pass

In [ ]:
def des_encryption(plaintext: int, key: int) -> int:
    ciphertext1 = permutate_bits(plaintext)
    ciphertext2 = feistel_encrypt(ciphertext1, key)
    ciphertext3 = permutate_bits(ciphertext2)

    return ciphertext3

In [ ]:
def test_des():
    example_plaintext = 0x02468ACEECA86420
    example_key = 0x0F1571C947D9E859
    expected_ciphertext = 0xDA02CE3A89ECAC3B

    generated_ciphertext = des_encryption(example_plaintext, example_key)

    print(expected_ciphertext == generated_ciphertext)